# First part of Zack Wilson's bootcamp [spark homework](https://github.com/DataExpert-io/data-engineer-handbook/blob/main/bootcamp/materials/3-spark-fundamentals/homework/homework.md).

[Pyspark documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.alias.html)

### Make imports and create spark session if necessary

In [102]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("Jupyter").getOrCreate()

### Disable autoBroadcast joins. We will explicitely do it

In [2]:
# spark.sparkContext.getConf().getAll()
print(f"Originally, autoBroadcastJoinThreshold is {spark.conf.get('spark.sql.autoBroadcastJoinThreshold')}")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
print(f"Now is {spark.conf.get('spark.sql.autoBroadcastJoinThreshold')}")

Originally, autoBroadcastJoinThreshold is 10485760b
Now is -1


### Add a few helpers

In [246]:
def create_df_from_csv(csv_name):
    '''
        All csvs are in /home/iceberg/data/
        Needs spark
    '''
    return spark \
        .read \
        .option("header","true") \
        .option("inferSchema", "true") \
        .csv(f"/home/iceberg/data/{csv_name}.csv")

def drop_table(table_name, schema_name="bootcamp"):
    spark.sql(f"drop table if exists {schema_name}.{table_name}")

def print_table_size(table_name, schema_name="bootcamp"):
    file_sizes_df = spark.sql(f"""
        select 
            file_path, 
            file_size_in_bytes 
        from 
            {schema_name}.{table_name}.files
    """)

    # file_sizes_df.show(truncate=False)
    
    total_size = file_sizes_df.agg({"file_size_in_bytes": "sum"}).collect()[0][0]
    print(f"Total table size: {total_size/1000000} Mb")

### Read the necessary datasets from csvs

In [11]:
medals = create_df_from_csv("medals")

maps = create_df_from_csv("maps")

match_details = create_df_from_csv("match_details")

matches = create_df_from_csv("matches")

medals_matches_players = create_df_from_csv("medals_matches_players")


In [12]:
for df in [medals, maps, match_details, matches, medals_matches_players]:
    print(f"{df.count()}")

183
40
151761
24025
755229


### Handle joins directly
* medals and maps are considerably smaller than the others => broadcast them when join
* create iceberg tables upon them in order to leverage bucket joins 

### Lets create tables and populate with their respective csvs

In [6]:
spark.sql("show tables in bootcamp").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
| bootcamp|match_details_buc...|      false|
| bootcamp|    matches_bucketed|      false|
| bootcamp|medals_matches_pl...|      false|
+---------+--------------------+-----------+



In [188]:
drop_table("match_details_bucketed")

match_details_bucketed_ddl = """
CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
    match_id STRING,
    player_gamertag STRING,
    player_total_kills INTEGER,
    player_total_deaths INTEGER
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""

spark.sql(match_details_bucketed_ddl)

match_details \
    .select(
        "match_id", "player_gamertag", "player_total_kills", "player_total_deaths"
    ) \
    .write.mode("overwrite") \
    .bucketBy(16, "match_id") \
    .saveAsTable("bootcamp.match_details_bucketed")

In [187]:
drop_table("matches_bucketed")

matches_bucketed_ddl = """
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
     match_id STRING,
     mapid STRING,
     is_team_game BOOLEAN,
     playlist_id STRING,
     completion_date TIMESTAMP
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id));
 """

spark.sql(matches_bucketed_ddl)

matches \
    .select(
        "match_id", "mapid", "is_team_game", "playlist_id", "completion_date"
    ) \
    .write.mode("overwrite") \
    .bucketBy(16, "match_id") \
    .saveAsTable("bootcamp.matches_bucketed")


In [186]:
drop_table("medals_matches_players_bucketed")

medals_matches_players_bucketed_ddl = """
CREATE TABLE IF NOT EXISTS bootcamp.medals_matches_players_bucketed (
    match_id STRING,
    medal_id STRING,
    player_gamertag STRING,
    count INTEGER
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""

spark.sql(medals_matches_players_bucketed_ddl)

medals_matches_players \
    .select(
        "match_id", "medal_id", "player_gamertag", "count"
    ) \
    .write.mode("overwrite") \
    .bucketBy(16, "match_id") \
    .saveAsTable("bootcamp.medals_matches_players_bucketed")


### Reselect them as tables

In [45]:
match_details_table = spark.sql("select * from bootcamp.match_details_bucketed")
matches_table = spark.sql("select * from bootcamp.matches_bucketed")
medals_matches_players_table = spark.sql("select * from bootcamp.medals_matches_players_bucketed")

In [18]:
spark.sql("select * from bootcamp.medals_matches_players_bucketed").count()

755229

### Answering questions from the homework
1. Which player averages the most kills per game?
2. Which playlist gets played the most?
3. Which map gets played the most?
4. Which map do players get the most Killing Spree medals on?

In the assignement, we are asked first to join all the dataframes and then answer questions but I feel like this would not be the more efficient. I do the join in order to comply but I'll answer questions differently.

WARNING : in the _medals_ dataset, some _medal\_id_ does not have a _name_ and their are duplicates in _name_ => the joined dataframe will have a row per _match\_id_, _player\_gamertag_, _medal\_id_ (we have to keep the last one to ensure uniqueness).

In [130]:
joined_df = match_details_table \
    .join(
        matches_table
        , matches_table.match_id == match_details_table.match_id
        , how="inner"
    ) \
    .join(
        medals_matches_players_table
        , [medals_matches_players_table.match_id == match_details_table.match_id
            , medals_matches_players_table.player_gamertag == match_details_table.player_gamertag]
        , how="inner"
    ) \
    .join(
        broadcast(maps)
        , maps.mapid == matches_table.mapid
        , how="inner"
    ) \
    .join(
        broadcast(medals)
        , medals.medal_id == medals_matches_players_table.medal_id
        , how="inner"
    ) \
    .select(
        match_details_table.match_id
        , match_details_table.player_gamertag
        , medals_matches_players_table.medal_id
        , maps.name.alias("map_name")
        , matches_table.playlist_id
        , match_details_table.player_total_kills
        , match_details_table.player_total_deaths
        , medals.name.alias("medal_name")
        , F.col("count").alias("medal_count")
    )


In [ ]:
# Top 5 avg kills player, limit should be deterministic
match_details_table \
    .groupBy("player_gamertag") \
    .agg(F.avg("player_total_kills").alias("avg_kills")) \
    .sort(F.desc("avg_kills")) \
    .limit(5) \
    .show()
    

+---------------+---------+
|player_gamertag|avg_kills|
+---------------+---------+
|   gimpinator14|    109.0|
|  I Johann117 I|     96.0|
|BudgetLegendary|     83.0|
|      GsFurreal|     75.0|
|   Sexy is Back|     73.0|
+---------------+---------+



In [ ]:
# Top 5 most played playlists. Did not found were to look for playlist_name, limit should be deterministic
matches_table \
    .groupBy("playlist_id") \
    .agg(F.count("match_id").alias("nb_played")) \
    .sort(F.desc("nb_played")) \
    .limit(5) \
    .show()

+--------------------+---------+
|         playlist_id|nb_played|
+--------------------+---------+
|f72e0ef0-7c4a-430...|     9350|
|2323b76a-db98-4e0...|     3244|
|892189e9-d712-4bd...|     2159|
|c98949ae-60a8-43d...|     1984|
|b50c4dc2-6c86-4d7...|     1462|
+--------------------+---------+



In [173]:
# Top 5 most played maps, limit should be deterministic
matches_table \
    .join(broadcast(maps)
          , matches_table.mapid == maps.mapid
    ) \
    .select(
        matches_table.mapid
        , maps.name.alias("map_name")
    ) \
    .groupBy("map_name") \
    .agg(F.count("mapid").alias("nb_played")) \
    .sort(F.desc("nb_played")) \
    .limit(5) \
    .show()

+--------------+---------+
|      map_name|nb_played|
+--------------+---------+
|Breakout Arena|     8587|
|        Empire|     1489|
|        Alpine|     1461|
|       The Rig|     1088|
|       Glacier|     1052|
+--------------+---------+



In [171]:
# Top 5 maps having the most Killing Spree medal, limit should be deterministic
joined_df \
    .where("medal_name='Killing Spree'") \
    .groupBy("map_name") \
    .agg(F.sum("medal_count").alias("sum_medals")) \
    .sort(F.desc("sum_medals")) \
    .limit(5) \
    .show()

+--------------+----------+
|      map_name|sum_medals|
+--------------+----------+
|Breakout Arena|      6738|
|        Alpine|      5359|
|       Glacier|      3402|
|        Empire|      2233|
|         Truth|      2061|
+--------------+----------+



### Find best keys to sort partitions in order to find the smallest total file size

In the homework, it is written to sort the aggregated dataset but I do it on the joined one because I feel like there was no single aggregated dataset

In [229]:
def print_file_size_from_sort_keys(df, sorting_keys=[], base_table_name="joined", schema_name="bootcamp"):
    table_name = base_table_name
    for key in sorting_keys: table_name += f"_{key}"
    print(f"table name is : {table_name}")
    df \
        .sortWithinPartitions(sorting_keys) \
        .write.mode("overwrite") \
        .saveAsTable(f"{schema_name}.{table_name}")
    print_table_size(table_name)

In [237]:
print_file_size_from_sort_keys(joined_df)

table name is : joined
+---------------------------------------------------------------------------------------------------+------------------+
|file_path                                                                                          |file_size_in_bytes|
+---------------------------------------------------------------------------------------------------+------------------+
|s3://warehouse/bootcamp/joined/data/00000-1381-3d948036-92d0-4183-9c5b-645032e21819-0-00001.parquet|556199            |
|s3://warehouse/bootcamp/joined/data/00001-1382-3d948036-92d0-4183-9c5b-645032e21819-0-00001.parquet|545976            |
|s3://warehouse/bootcamp/joined/data/00002-1383-3d948036-92d0-4183-9c5b-645032e21819-0-00001.parquet|542160            |
|s3://warehouse/bootcamp/joined/data/00003-1384-3d948036-92d0-4183-9c5b-645032e21819-0-00001.parquet|544373            |
|s3://warehouse/bootcamp/joined/data/00004-1385-3d948036-92d0-4183-9c5b-645032e21819-0-00001.parquet|535833            |
|s3://war

In [238]:
print_file_size_from_sort_keys(joined_df, ["match_id"])

table name is : joined_match_id
+------------------------------------------------------------------------------------------------------------+------------------+
|file_path                                                                                                   |file_size_in_bytes|
+------------------------------------------------------------------------------------------------------------+------------------+
|s3://warehouse/bootcamp/joined_match_id/data/00000-1417-e74683fd-2f5a-47b3-9281-ab146d6aeda9-0-00001.parquet|556199            |
|s3://warehouse/bootcamp/joined_match_id/data/00001-1418-e74683fd-2f5a-47b3-9281-ab146d6aeda9-0-00001.parquet|545976            |
|s3://warehouse/bootcamp/joined_match_id/data/00002-1419-e74683fd-2f5a-47b3-9281-ab146d6aeda9-0-00001.parquet|542160            |
|s3://warehouse/bootcamp/joined_match_id/data/00003-1420-e74683fd-2f5a-47b3-9281-ab146d6aeda9-0-00001.parquet|544373            |
|s3://warehouse/bootcamp/joined_match_id/data/00004-1421-e

In [239]:
print_file_size_from_sort_keys(joined_df, ["match_id", "player_gamertag"])

table name is : joined_match_id_player_gamertag
+----------------------------------------------------------------------------------------------------------------------------+------------------+
|file_path                                                                                                                   |file_size_in_bytes|
+----------------------------------------------------------------------------------------------------------------------------+------------------+
|s3://warehouse/bootcamp/joined_match_id_player_gamertag/data/00000-1453-cfa20978-c230-4e69-b062-bace6a86ae39-0-00001.parquet|556199            |
|s3://warehouse/bootcamp/joined_match_id_player_gamertag/data/00001-1454-cfa20978-c230-4e69-b062-bace6a86ae39-0-00001.parquet|545976            |
|s3://warehouse/bootcamp/joined_match_id_player_gamertag/data/00002-1455-cfa20978-c230-4e69-b062-bace6a86ae39-0-00001.parquet|542160            |
|s3://warehouse/bootcamp/joined_match_id_player_gamertag/data/00003-1456-cfa

In [240]:
print_file_size_from_sort_keys(joined_df, ["match_id", "player_gamertag", "map_name"])

table name is : joined_match_id_player_gamertag_map_name
+-------------------------------------------------------------------------------------------------------------------------------------+------------------+
|file_path                                                                                                                            |file_size_in_bytes|
+-------------------------------------------------------------------------------------------------------------------------------------+------------------+
|s3://warehouse/bootcamp/joined_match_id_player_gamertag_map_name/data/00000-1489-6900f47d-eadc-42c7-bff7-03ac7d8e4117-0-00001.parquet|556199            |
|s3://warehouse/bootcamp/joined_match_id_player_gamertag_map_name/data/00001-1490-6900f47d-eadc-42c7-bff7-03ac7d8e4117-0-00001.parquet|545976            |
|s3://warehouse/bootcamp/joined_match_id_player_gamertag_map_name/data/00002-1491-6900f47d-eadc-42c7-bff7-03ac7d8e4117-0-00001.parquet|542160            |
|s3://warehou

In [241]:
print_file_size_from_sort_keys(joined_df, ["match_id", "player_gamertag", "map_name", "playlist_id"])

table name is : joined_match_id_player_gamertag_map_name_playlist_id
+-------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|file_path                                                                                                                                        |file_size_in_bytes|
+-------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|s3://warehouse/bootcamp/joined_match_id_player_gamertag_map_name_playlist_id/data/00000-1525-1570bb5b-35af-4d3e-907f-fc250a3dbd31-0-00001.parquet|556199            |
|s3://warehouse/bootcamp/joined_match_id_player_gamertag_map_name_playlist_id/data/00001-1526-1570bb5b-35af-4d3e-907f-fc250a3dbd31-0-00001.parquet|545976            |
|s3://warehouse/bootcamp/joined_match_id_player_gamertag_map_name_playlist_id/data/00002-1527-15

In [242]:
print_file_size_from_sort_keys(joined_df, ["map_name", "playlist_id"])

table name is : joined_map_name_playlist_id
+------------------------------------------------------------------------------------------------------------------------+------------------+
|file_path                                                                                                               |file_size_in_bytes|
+------------------------------------------------------------------------------------------------------------------------+------------------+
|s3://warehouse/bootcamp/joined_map_name_playlist_id/data/00000-1561-a713a7f9-9cac-46b3-a6f1-b3d0b44107ba-0-00001.parquet|518156            |
|s3://warehouse/bootcamp/joined_map_name_playlist_id/data/00001-1562-a713a7f9-9cac-46b3-a6f1-b3d0b44107ba-0-00001.parquet|517032            |
|s3://warehouse/bootcamp/joined_map_name_playlist_id/data/00002-1563-a713a7f9-9cac-46b3-a6f1-b3d0b44107ba-0-00001.parquet|513594            |
|s3://warehouse/bootcamp/joined_map_name_playlist_id/data/00003-1564-a713a7f9-9cac-46b3-a6f1-b3d0b44107b

In [243]:
print_file_size_from_sort_keys(joined_df, ["playlist_id", "map_name"])

table name is : joined_playlist_id_map_name
+------------------------------------------------------------------------------------------------------------------------+------------------+
|file_path                                                                                                               |file_size_in_bytes|
+------------------------------------------------------------------------------------------------------------------------+------------------+
|s3://warehouse/bootcamp/joined_playlist_id_map_name/data/00000-1597-e8289d61-f286-4b40-9a34-22960f095697-0-00001.parquet|517441            |
|s3://warehouse/bootcamp/joined_playlist_id_map_name/data/00001-1598-e8289d61-f286-4b40-9a34-22960f095697-0-00001.parquet|515904            |
|s3://warehouse/bootcamp/joined_playlist_id_map_name/data/00002-1599-e8289d61-f286-4b40-9a34-22960f095697-0-00001.parquet|508226            |
|s3://warehouse/bootcamp/joined_playlist_id_map_name/data/00003-1600-e8289d61-f286-4b40-9a34-22960f09569

In [244]:
print_file_size_from_sort_keys(joined_df, ["playlist_id"])

table name is : joined_playlist_id
+---------------------------------------------------------------------------------------------------------------+------------------+
|file_path                                                                                                      |file_size_in_bytes|
+---------------------------------------------------------------------------------------------------------------+------------------+
|s3://warehouse/bootcamp/joined_playlist_id/data/00000-1633-ae92856e-278e-4679-ab28-346dcd56b072-0-00001.parquet|528154            |
|s3://warehouse/bootcamp/joined_playlist_id/data/00001-1634-ae92856e-278e-4679-ab28-346dcd56b072-0-00001.parquet|523819            |
|s3://warehouse/bootcamp/joined_playlist_id/data/00002-1635-ae92856e-278e-4679-ab28-346dcd56b072-0-00001.parquet|514774            |
|s3://warehouse/bootcamp/joined_playlist_id/data/00003-1636-ae92856e-278e-4679-ab28-346dcd56b072-0-00001.parquet|525455            |
|s3://warehouse/bootcamp/joined_pl

In [245]:
print_file_size_from_sort_keys(joined_df, ["map_name"])

table name is : joined_map_name
+------------------------------------------------------------------------------------------------------------+------------------+
|file_path                                                                                                   |file_size_in_bytes|
+------------------------------------------------------------------------------------------------------------+------------------+
|s3://warehouse/bootcamp/joined_map_name/data/00000-1669-dd0ac11a-d8d2-45be-b7e3-07cf99b34452-0-00001.parquet|531473            |
|s3://warehouse/bootcamp/joined_map_name/data/00001-1670-dd0ac11a-d8d2-45be-b7e3-07cf99b34452-0-00001.parquet|526259            |
|s3://warehouse/bootcamp/joined_map_name/data/00002-1671-dd0ac11a-d8d2-45be-b7e3-07cf99b34452-0-00001.parquet|526445            |
|s3://warehouse/bootcamp/joined_map_name/data/00003-1672-dd0ac11a-d8d2-45be-b7e3-07cf99b34452-0-00001.parquet|526395            |
|s3://warehouse/bootcamp/joined_map_name/data/00004-1673-d